# ARIMA

**Assumptions:**
- Stationary time series
    - If it's not do differences until it is

In [1]:
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd

## Prepare the Data

## Splits

## Training a Baseline

## Hyper-Parameter Tuning

## Evaluation